In [1]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = "sagemaker/pytorch-toneclone"

role = sagemaker.get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
!pygmentize spectrogramCNN_sagemaker.py

import numpy as np
import h5py
import pandas as pd
import argparse
import json
import logging
import os
import sys
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

#import sagemaker_containers
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.utils.data.distributed
from torchvision import datasets, transforms
from sklearn.metrics import classification_report

class SpectrogramDataset(Dataset):
    """
    Custom dataset for spectrogram data.

    When the Spectrogram dataset is used to create a dataloader object, the
    dataloader consists of batches of spectrograms and their corresponding labels.
    Here is info on the shape of the spectrogram and label objects in each batch:

    Spectrogram Tensor Dimensions in Batch - (32, 1, 128, 626)
        Batch Size: 32
        Channels: 1 - Think of it as a grayscale 

In [3]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point="spectrogramCNN_sagemaker.py",
    role=role,
    py_version="py38",
    framework_version="1.11.0",
    instance_count=2,
    instance_type="ml.g5.12xlarge",
    hyperparameters={"epochs": 1},
)

In [4]:
estimator.fit({"training": 's3://toneclone-bucket/Training Data/',"validation": 's3://toneclone-bucket/Validation Data/'})

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2025-02-15-19-50-01-824


2025-02-15 19:50:03 Starting - Starting the training job...
2025-02-15 19:50:35 Downloading - Downloading input data....................................................................................
2025-02-15 20:04:38 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2025-02-15 15:04:43,688 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2025-02-15 15:04:43,690 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-02-15 15:04:43,692 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2025-02-15 15:04:43,703 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2025-02-15 15:04:43,711 sagemaker_pytorch_container.training INFO     Invoking user training script.
2025-02-15 15:04:44,043 sagemaker-training-toolkit

UnexpectedStatusException: Error for Training job pytorch-training-2025-02-15-19-50-01-824: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
ExitCode 1
ErrorMessage "NameError: name 'logger' is not defined"
Command "/opt/conda/bin/python3.8 spectrogramCNN_sagemaker.py --epochs 1", exit code: 1